In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
import os
import pandas as pd
import re
import pickle
from google.colab import drive

# **🔗 Mount Google Drive**
drive.mount('/content/drive')
drive_save_path = "/content/drive/MyDrive/chatbot/"
os.makedirs(drive_save_path, exist_ok=True)

# **⚡ Enable mixed precision**
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# **📝 Text Preprocessing Function**
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'s", " is", text)
    text = re.sub(r"'d", " would", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'t", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'m", " am", text)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    return text.strip()

# **📥 Load Skyrim Dialogue Dataset**
def load_skyrim_dialogue_dataset(file_path):
    df = pd.read_csv(file_path)
    player_inputs = df['Player Input'].astype(str).apply(preprocess_text).tolist()
    npc_responses = df['NPC Response'].astype(str).apply(preprocess_text).tolist()
    return player_inputs, npc_responses

file_path = "/content/drive/MyDrive/chatbot/skyrim_dialogue_dataset_10000V2.csv"
player_inputs, npc_responses = load_skyrim_dialogue_dataset(file_path)

# **📌 Tokenization & Padding**
max_len = 50

# Tokenizer
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(player_inputs + npc_responses)
vocab_size = len(tokenizer.word_index) + 1
print(f"✅ Vocabulary Size: {vocab_size}")

# Save tokenizer
tokenizer_path = os.path.join(drive_save_path, "tokenizer.pkl")
with open(tokenizer_path, "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Convert to sequences
input_padded = pad_sequences(input_sequences, maxlen=max_len, padding='post')
response_padded = pad_sequences(response_sequences, maxlen=max_len, padding='post')
# **🧪 Train-Test Split**
X_train, X_val, y_train, y_val = train_test_split( input_padded, response_padded, test_size=0.2, random_state=42)
# **🛠️ LSTM Model Architecture**
embedding_dim = 256  # Increased embedding dimension
lstm_units = 512  # Increased LSTM units
dropout_rate = 0.3

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    Bidirectional(LSTM(lstm_units, return_sequences=True)),
    Dropout(dropout_rate),
    Bidirectional(LSTM(lstm_units, return_sequences=True)),
    Dropout(dropout_rate),
    Dense(vocab_size, activation='softmax')
])


# **⚙️ Compile Model**
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.002), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# **📌 Custom Callback to Save Tokenizer**
class SaveTokenizerCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        with open(tokenizer_path, "wb") as handle:
            pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(f"✅ Tokenizer saved after epoch {epoch + 1}")

# **🛡️ Callbacks**
best_model_path = os.path.join(drive_save_path, "skyrim_chatbot_best.keras")
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    ModelCheckpoint(best_model_path, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=0.0001),
    SaveTokenizerCallback()
]

# **🎯 Train Model**
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=callbacks
)

# **✅ Save Final Model**
final_model_path = os.path.join(drive_save_path, "skyrim_chatbot_final.keras")
model.save(final_model_path)
print(f"✅ Model saved as {final_model_path}")

# **🗣️ Generate NPC Response Function**
def generate_response(player_input, model, tokenizer, max_sequence_length, temperature=0.8):
    input_seq = tokenizer.texts_to_sequences([player_input])
    input_seq = pad_sequences(input_seq, maxlen=max_sequence_length, padding='post')

    predicted_seq = model.predict(input_seq)[0]
    predicted_seq = np.log(predicted_seq + 1e-8) / temperature
    predicted_seq = np.exp(predicted_seq) / np.sum(np.exp(predicted_seq))
    sampled_indices = [np.random.choice(len(seq), p=seq) for seq in predicted_seq]

    response = tokenizer.sequences_to_texts([sampled_indices])[0]
    return response if response else "I am not sure, traveler."

# **🤖 Example Chatbot Interaction**
print("Welcome to the Skyrim Chatbot! Type 'exit' to leave.")
while True:
    user_input = input("You: ").strip().lower()
    if user_input in ["exit", "quit"]:
        print("NPC: Farewell, traveler!")
        break

    response = generate_response(user_input, model, tokenizer, max_sequence_length)
    print(f"NPC: {response}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Vocabulary Size: 3594


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.7003 - loss: 3.1384✅ Tokenizer saved after epoch 1
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 216ms/step - accuracy: 0.7005 - loss: 3.1320 - val_accuracy: 0.7403 - val_loss: 1.9103 - learning_rate: 0.0020
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.7390 - loss: 1.8978✅ Tokenizer saved after epoch 2
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 160ms/step - accuracy: 0.7390 - loss: 1.8977 - val_accuracy: 0.7411 - val_loss: 1.8682 - learning_rate: 0.0020
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.7388 - loss: 1.8430✅ Tokenizer saved after epoch 3
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 158ms/step - accuracy: 0.7388 - loss: 1.8429 - val_accuracy: 0.7416 - val_loss: 1.8260 - learning_rate: 0.0020
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.7394 - loss: 1.7905✅ Tokenizer saved after epoch 4
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 152ms/step - accuracy: 0.7394 - loss: 1.7903 - val_accuracy

<ipython-input-3-706784b43899>:116: RuntimeWarning: divide by zero encountered in log
  predicted_seq = np.log(predicted_seq + 1e-8) / temperature


ValueError: probabilities do not sum to 1

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import pickle
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import drive  # Mount Google Drive

# **🔗 Mount Google Drive**
drive.mount('/content/drive')

# **📁 Define Google Drive Save Path**
drive_save_path = "/content/drive/MyDrive/chatbot/"
best_model_path = os.path.join(drive_save_path, "skyrim_chatbot_best.keras")
updated_model_path = os.path.join(drive_save_path, "skyrim_chatbot_best.keras")
tokenizer_path = os.path.join(drive_save_path, "tokenizer.pkl")

# **📌 Load the Saved Model**
if os.path.exists(best_model_path):
    model = tf.keras.models.load_model(best_model_path)
    print(f"✅ Model loaded from {best_model_path}")
else:
    raise FileNotFoundError(f"❌ Model file not found at {best_model_path}")

# **📌 Load Tokenizer**
if os.path.exists(tokenizer_path):
    with open(tokenizer_path, "rb") as handle:
        tokenizer = pickle.load(handle)
    print(f"✅ Tokenizer loaded from {tokenizer_path}")
else:
    raise FileNotFoundError(f"❌ Tokenizer file not found at {tokenizer_path}")

# **📝 Text Preprocessing Function**
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'s", " is", text)
    text = re.sub(r"'d", " would", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'t", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'m", " am", text)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    return text.strip()

# **📂 Load Skyrim Dialogue Dataset**
file_path = "/content/drive/MyDrive/chatbot/dataset2796.csv"
df = pd.read_csv(file_path)

player_inputs = df['Player Input'].astype(str).apply(preprocess_text).tolist()
npc_responses = df['NPC Response'].astype(str).apply(preprocess_text).tolist()

# **Tokenize & Pad Sequences**
max_sequence_length = 30  # Same as previous training

input_sequences = pad_sequences(tokenizer.texts_to_sequences(player_inputs), maxlen=max_sequence_length, padding='post')
response_sequences = pad_sequences(tokenizer.texts_to_sequences(npc_responses), maxlen=max_sequence_length, padding='post')

vocab_size = len(tokenizer.word_index) + 1
print(f"✅ Vocabulary Size: {vocab_size}")

# **Convert responses to numpy array for training**
y_train = np.array(response_sequences)

# **🧪 Split dataset into train & test**
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(input_sequences, y_train, test_size=0.2, random_state=42)

# **⚙️ Recompile Model with Optimizer & Loss**
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.002), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# **🛡️ Custom Callback to Save Model & Tokenizer After Each Epoch**
class SaveModelAndTokenizer(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Save model
        model.save(updated_model_path)

        # Save tokenizer
        with open(tokenizer_path, "wb") as handle:
            pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

        print(f"✅ Epoch {epoch+1}: Model & Tokenizer saved!")

# **🛡️ Callbacks for Training**
callbacks = [
    SaveModelAndTokenizer(),  # Custom callback to save both model & tokenizer
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
]

# **🎯 Retrain Model**
try:
    epochs = 50  # Additional training
    batch_size = 64

    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        callbacks=callbacks
    )

    print(f"✅ Retraining complete! Model & Tokenizer saved after every epoch.")

except KeyboardInterrupt:
    print("\n⚠️ Training interrupted! Saving latest model before exiting...")
    model.save(os.path.join(drive_save_path, "skyrim_chatbot_latest.keras"))
    print(f"✅ Latest model saved as skyrim_chatbot_latest.keras")

vocab_size = len(tokenizer.word_index) + 1
print(f"✅ Vocabulary Size: {vocab_size}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model loaded from /content/drive/MyDrive/chatbot/skyrim_chatbot_best.keras
✅ Tokenizer loaded from /content/drive/MyDrive/chatbot/tokenizer.pkl
✅ Vocabulary Size: 3591
Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - accuracy: 0.8535 - loss: 0.5860✅ Epoch 1: Model & Tokenizer saved!
35/35 ━━━━━━━━━━━━━━━━━━━━ 20s 290ms/step - accuracy: 0.8533 - loss: 0.5868 - val_accuracy: 0.7412 - val_loss: 2.1781 - learning_rate: 0.0020
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - accuracy: 0.8569 - loss: 0.5620✅ Epoch 2: Model & Tokenizer saved!
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 251ms/step - accuracy: 0.8568 - loss: 0.5625 - val_accuracy: 0.7379 - val_loss: 2.1798 - learning_rate: 0.0020
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.8666 - loss: 0.5278✅ Epoch 3: Model & Tokenizer saved!
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 221ms/step - accuracy: 0.

In [ ]:
import json
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import random
from google.colab import drive  # Mount Google Drive

# **🔗 Mount Google Drive**
drive.mount('/content/drive')

# **📁 Define Google Drive Path**
drive_save_path = "/content/drive/MyDrive/chatbot/"
model_path = os.path.join(drive_save_path, "skyrim_chatbot_ea.keras")
tokenizer_path = os.path.join(drive_save_path, "tokenizer.pkl")

# **📌 Load the Updated Model**
if os.path.exists(model_path):
    model = tf.keras.models.load_model(model_path)
    print(f"✅ Model loaded from {model_path}")
else:
    raise FileNotFoundError(f"❌ Model file not found at {model_path}")

# **📌 Load the Tokenizer**
if os.path.exists(tokenizer_path):
    with open(tokenizer_path, "rb") as handle:
        tokenizer = pickle.load(handle)
    print(f"✅ Tokenizer loaded from {tokenizer_path}")
else:
    raise FileNotFoundError(f"❌ Tokenizer file not found at {tokenizer_path}")

# **Response Generation Function (Fixed)**
def generate_response(input_text, max_response_length=30, temperature=0.2, top_k=5):
    max_sequence_length = 30  # Must match training

    # **Tokenize and pad the input text**
    input_seq = pad_sequences(tokenizer.texts_to_sequences([input_text]), maxlen=max_sequence_length, padding='post')

    # **Predict the response sequence**
    prediction = model.predict(input_seq, verbose=0)  # Shape: (1, max_sequence_length, vocab_size)

    # **Ensure response generation starts correctly**
    response_text = []
    previous_words = set()  # Track previous words to avoid repetition

    for i in range(max_response_length):
        word_probs = prediction[0, i]  # Extract probabilities for the i-th word

        # **Temperature scaling for controlled randomness**
        word_probs = np.exp(word_probs / temperature)
        word_probs /= np.sum(word_probs)  # Normalize probabilities

        # **Select the top-k words (Beam Search)**
        top_indices = np.argsort(word_probs)[-top_k:]  # Get top-k indices
        word_idx = np.random.choice(top_indices, p=word_probs[top_indices] / np.sum(word_probs[top_indices]))

        # **Handle unknown tokens & repetition**
        if word_idx not in tokenizer.index_word:
            continue  # Skip unrecognized words

        word = tokenizer.index_word[word_idx]

        if word in previous_words or word in ["<OOV>", "<UNK>", "<PAD>", "<END>"]:
            continue  # Prevents repeating the same word or invalid tokens

        response_text.append(word)
        previous_words.add(word)  # Track words used

        # **Break if a complete response is formed**
        if word in [".", "?", "!"]:
            break

    # **Format the final response**
    response = " ".join(response_text).strip()

    # **Fallback Response if Model Fails**
    if not response:
        fallback_responses = [
            "I am not sure about that traveler, but perhaps the answer lies elsewhere.",
            "That is a question for the wise, not a mere wanderer like me.",
            "Perhaps the Divines have the answer you seek.",
            "That is a tale lost to time, traveler."
        ]
        response = np.random.choice(fallback_responses)  # Random fallback for variety

    return response

# **Chatbot interaction loop**
try:
    print("Welcome to the Elder Scrolls NPC Chatbot! Type 'exit' or 'quit' to end the conversation.")
    while True:
        user_input = input("You: ").strip().lower()

        # **Exit conditions**
        if user_input in ["exit", "quit"]:
            print("NPC: Farewell, traveler. May the blessings of the Tribunal be with you!")
            break

        # **Generate and display NPC response**
        response = generate_response(user_input)
        print(f"NPC: {response}")

except KeyboardInterrupt:
    print("\nNPC: Farewell, traveler. May the blessings of the Tribunal be with you!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


TypeError: <class 'keras.src.models.functional.Functional'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {}, 'registered_name': 'Functional', 'build_config': {'input_shape': None}, 'compile_config': {'optimizer': {'module': 'keras.optimizers', 'class_name': 'Adam', 'config': {'name': 'adam', 'learning_rate': {'module': 'keras.optimizers.schedules', 'class_name': 'ExponentialDecay', 'config': {'initial_learning_rate': 0.002, 'decay_steps': 100000, 'decay_rate': 0.96, 'staircase': True, 'name': 'ExponentialDecay'}, 'registered_name': None}, 'weight_decay': None, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_ac

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
import os
import pandas as pd
import re
import pickle
from google.colab import drive
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# **🔗 Mount Google Drive**
drive.mount('/content/drive')
drive_save_path = "/content/drive/MyDrive/chatbot/"
os.makedirs(drive_save_path, exist_ok=True)

# **⚡ Enable mixed precision**
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# **📝 Text Preprocessing Function**
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'s", " is", text)
    text = re.sub(r"'d", " would", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'t", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'m", " am", text)
    text = re.sub(r"[^a-zAZ0-9\s]", "", text)
    return text.strip()

# **📥 Load Skyrim Dialogue Dataset**
def load_skyrim_dialogue_dataset(file_path):
    df = pd.read_csv(file_path)
    player_inputs = df['Player Input'].astype(str).apply(preprocess_text).tolist()
    npc_responses = df['NPC Response'].astype(str).apply(preprocess_text).tolist()
    return player_inputs, npc_responses

file_path = "/content/drive/MyDrive/chatbot/skyrim_dialogue_dataset_10000V2.csv"
player_inputs, npc_responses = load_skyrim_dialogue_dataset(file_path)

# **📌 Tokenization & Padding**
max_sequence_length = 30

# Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(player_inputs + npc_responses)
vocab_size = len(tokenizer.word_index) + 1
print(f"✅ Vocabulary Size: {vocab_size}")

# Save tokenizer
tokenizer_path = os.path.join(drive_save_path, "tokenizer.pkl")
with open(tokenizer_path, "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Convert to sequences
input_sequences = pad_sequences(tokenizer.texts_to_sequences(player_inputs), maxlen=max_sequence_length, padding='post')
response_sequences = pad_sequences(tokenizer.texts_to_sequences(npc_responses), maxlen=max_sequence_length, padding='post')
y_train = np.array(response_sequences, dtype=np.int32)

# **🧪 Train-Test Split**
X_train, X_test, y_train, y_test = train_test_split(input_sequences, y_train, test_size=0.2, random_state=42)

# **🛠️ LSTM Model Architecture with Attention Layer**
embedding_dim = 256
lstm_units = 512  # Increased LSTM units for more capacity

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)),
    tf.keras.layers.Attention(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size, activation='softmax', dtype='float32'))
])

# **⚙️ Compile Model with Learning Rate Schedule**
lr_schedule = ExponentialDecay(initial_learning_rate=0.002, decay_steps=100000, decay_rate=0.96, staircase=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# **📌 Custom Callback to Save Tokenizer**
class SaveTokenizerCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        with open(tokenizer_path, "wb") as handle:
            pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(f"✅ Tokenizer saved after epoch {epoch + 1}")

# **🛡️ Callbacks**
best_model_path = os.path.join(drive_save_path, "skyrim_chatbot_best.keras")
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    ModelCheckpoint(best_model_path, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=0.0001),
    SaveTokenizerCallback()
]

# **🎯 Train Model**
history = model.fit(
    X_train, y_train,
    epochs=100,  # Increased epochs for better training
    batch_size=128,  # Larger batch size for stability
    validation_data=(X_test, y_test),
    callbacks=callbacks
)

# **✅ Save Final Model**
final_model_path = os.path.join(drive_save_path, "skyrim_chatbot_final.keras")
model.save(final_model_path)
print(f"✅ Model saved as {final_model_path}")

# **🗣️ Generate NPC Response Function**
def generate_response(player_input, model, tokenizer, max_sequence_length, temperature=0.8):
    input_seq = tokenizer.texts_to_sequences([player_input])
    input_seq = pad_sequences(input_seq, maxlen=max_sequence_length, padding='post')

    predicted_seq = model.predict(input_seq)[0]
    predicted_seq = np.log(predicted_seq + 1e-8) / temperature
    predicted_seq = np.exp(predicted_seq) / np.sum(np.exp(predicted_seq))
    sampled_indices = [np.random.choice(len(seq), p=seq) for seq in predicted_seq]

    response = tokenizer.sequences_to_texts([sampled_indices])[0]
    return response if response else "I am not sure, traveler."

# **🤖 Example Chatbot Interaction**
print("Welcome to the Skyrim Chatbot! Type 'exit' to leave.")
while True:
    user_input = input("You: ").strip().lower()
    if user_input in ["exit", "quit"]:
        print("NPC: Farewell, traveler!")
        break

    response = generate_response(user_input, model, tokenizer, max_sequence_length)
    print(f"NPC: {response}")

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
import os
import pandas as pd
import re
import pickle
from google.colab import drive
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# **🔗 Mount Google Drive**
drive.mount('/content/drive')
drive_save_path = "/content/drive/MyDrive/chatbot/"
os.makedirs(drive_save_path, exist_ok=True)

# **⚡ Enable mixed precision**
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# **📝 Text Preprocessing Function**
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'s", " is", text)
    text = re.sub(r"'d", " would", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'t", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'m", " am", text)
    text = re.sub(r"[^a-zAZ0-9\s]", "", text)
    return text.strip()

# **📥 Load Skyrim Dialogue Dataset**
def load_skyrim_dialogue_dataset(file_path):
    df = pd.read_csv(file_path)
    player_inputs = df['Player Input'].astype(str).apply(preprocess_text).tolist()
    npc_responses = df['NPC Response'].astype(str).apply(preprocess_text).tolist()
    return player_inputs, npc_responses

file_path = "/content/drive/MyDrive/chatbot/skyrim_dialogue_dataset_10000V2.csv"
player_inputs, npc_responses = load_skyrim_dialogue_dataset(file_path)

# **📌 Tokenization & Padding**
max_sequence_length = 30

# Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(player_inputs + npc_responses)
vocab_size = len(tokenizer.word_index) + 1
print(f"✅ Vocabulary Size: {vocab_size}")

# Save tokenizer
tokenizer_path = os.path.join(drive_save_path, "tokenizer.pkl")
with open(tokenizer_path, "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Convert to sequences
input_sequences = pad_sequences(tokenizer.texts_to_sequences(player_inputs), maxlen=max_sequence_length, padding='post')
response_sequences = pad_sequences(tokenizer.texts_to_sequences(npc_responses), maxlen=max_sequence_length, padding='post')
y_train = np.array(response_sequences, dtype=np.int32)

# **🧪 Train-Test Split**
X_train, X_test, y_train, y_test = train_test_split(input_sequences, y_train, test_size=0.2, random_state=42)

# **🛠️ LSTM Model Architecture with Attention Layer**
embedding_dim = 256
lstm_units = 512  # Increased LSTM units for more capacity

# LSTM Model with Attention
inputs = tf.keras.layers.Input(shape=(max_len,))
embed = tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=256)(inputs)
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True, dropout=0.3))(embed)
attention = tf.keras.layers.Attention()([lstm, lstm])
output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(tokenizer.word_index) + 1, activation='softmax', dtype='float32'))(attention)

model = tf.keras.models.Model(inputs, output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(0.002, 100000, 0.96, staircase=True)),
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(os.path.join(save_path, "best_model.keras"), save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=0.0001)
])


# **📌 Custom Callback to Save Tokenizer**
class SaveTokenizerCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        with open(tokenizer_path, "wb") as handle:
            pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(f"✅ Tokenizer saved after epoch {epoch + 1}")

# **🛡️ Callbacks**
best_model_path = os.path.join(drive_save_path, "skyrim_chatbot_best.keras")
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    ModelCheckpoint(best_model_path, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=0.0001),
    SaveTokenizerCallback()
]

# **🎯 Train Model**
history = model.fit(
    X_train, y_train,
    epochs=100,  # Increased epochs for better training
    batch_size=128,  # Larger batch size for stability
    validation_data=(X_test, y_test),
    callbacks=callbacks
)

# **✅ Save Final Model**
final_model_path = os.path.join(drive_save_path, "skyrim_chatbot_final.keras")
model.save(final_model_path)
print(f"✅ Model saved as {final_model_path}")

# **🗣️ Generate NPC Response Function**
def generate_response(player_input, model, tokenizer, max_sequence_length, temperature=0.3):
    input_seq = tokenizer.texts_to_sequences([player_input])
    input_seq = pad_sequences(input_seq, maxlen=max_sequence_length, padding='post')

    predicted_seq = model.predict(input_seq)[0]
    predicted_seq = np.log(predicted_seq + 1e-8) / temperature
    predicted_seq = np.exp(predicted_seq) / np.sum(np.exp(predicted_seq))
    sampled_indices = [np.random.choice(len(seq), p=seq) for seq in predicted_seq]

    response = tokenizer.sequences_to_texts([sampled_indices])[0]
    return response if response else "I am not sure, traveler."

# **🤖 Example Chatbot Interaction**
print("Welcome to the Skyrim Chatbot! Type 'exit' to leave.")
while True:
    user_input = input("You: ").strip().lower()
    if user_input in ["exit", "quit"]:
        print("NPC: Farewell, traveler!")
        break

    response = generate_response(user_input, model, tokenizer, max_sequence_length)
    print(f"NPC: {response}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Vocabulary Size: 3593
Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - accuracy: 0.6792 - loss: 3.4495✅ Tokenizer saved after epoch 1
63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 254ms/step - accuracy: 0.6798 - loss: 3.4364 - val_accuracy: 0.7345 - val_loss: 2.0273 - learning_rate: 0.0020
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.7350 - loss: 1.9882✅ Tokenizer saved after epoch 2
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 250ms/step - accuracy: 0.7351 - loss: 1.9879 - val_accuracy: 0.7403 - val_loss: 1.9139 - learning_rate: 0.0020
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - accuracy: 0.7403 - loss: 1.8984✅ Tokenizer saved after epoch 3
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 251ms/step - accuracy: 0.7403 - loss: 1.8983 - val_accuracy: 0.7410 - val_loss: 1.8676 - learning_rate: 0.0020
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - accuracy: 0.

TypeError: This optimizer was created with a `LearningRateSchedule` object as its `learning_rate` constructor argument, hence its learning rate is not settable. If you need the learning rate to be settable, you should instantiate the optimizer with a float `learning_rate` argument.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import pickle
from google.colab import drive
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback

# **🔗 Mount Google Drive**
drive.mount('/content/drive')
drive_save_path = "/content/drive/MyDrive/chatbot/"

# **📌 Load Tokenizer**
tokenizer_path = os.path.join(drive_save_path, "tokenizer.pkl")
with open(tokenizer_path, "rb") as handle:
    tokenizer = pickle.load(handle)

max_sequence_length = 30  # Ensure it matches the original model

# **📌 Load Saved Model**
best_model_path = os.path.join(drive_save_path, "skyrim_chatbot_best.keras")
model = tf.keras.models.load_model(best_model_path)

# **📥 Load Skyrim Dialogue Dataset**
def load_skyrim_dialogue_dataset(file_path):
    import pandas as pd
    import re

    def preprocess_text(text):
        text = text.lower()
        text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
        return text.strip()

    df = pd.read_csv(file_path)
    player_inputs = df['Player Input'].astype(str).apply(preprocess_text).tolist()
    npc_responses = df['NPC Response'].astype(str).apply(preprocess_text).tolist()
    return player_inputs, npc_responses

file_path = "/content/drive/MyDrive/chatbot/skyrim_dialogue_dataset_10000v1.1.csv"
player_inputs, npc_responses = load_skyrim_dialogue_dataset(file_path)

# **📌 Convert Text Data to Sequences**
input_sequences = pad_sequences(tokenizer.texts_to_sequences(player_inputs), maxlen=max_sequence_length, padding='post')
response_sequences = pad_sequences(tokenizer.texts_to_sequences(npc_responses), maxlen=max_sequence_length, padding='post')
y_train = np.array(response_sequences, dtype=np.int32)

# **🧪 Train-Test Split**
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(input_sequences, y_train, test_size=0.2, random_state=42)

# **📌 Callbacks for Training**
class SaveTokenizerCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        with open(tokenizer_path, "wb") as handle:
            pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(f"✅ Tokenizer saved after epoch {epoch + 1}")

# **Recompile Model Without Resetting Learning Rate**
model.compile(optimizer=model.optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint(best_model_path, save_best_only=True),
    SaveTokenizerCallback()
]

# **🎯 Continue Training**
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=callbacks
)

# **✅ Save Updated Model**
final_model_path = os.path.join(drive_save_path, "skyrim_chatbot_best.keras")
model.save(final_model_path)
print(f"✅ Retrained Model saved as {final_model_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - accuracy: 0.8365 - loss: 0.8239✅ Tokenizer saved after epoch 1
63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 265ms/step - accuracy: 0.8364 - loss: 0.8243 - val_accuracy: 0.8464 - val_loss: 0.8135
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - accuracy: 0.8349 - loss: 0.8327✅ Tokenizer saved after epoch 2
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 257ms/step - accuracy: 0.8349 - loss: 0.8327 - val_accuracy: 0.8480 - val_loss: 0.8131
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step - accuracy: 0.8379 - loss: 0.8120✅ Tokenizer saved after epoch 3
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 248ms/step - accuracy: 0.8379 - loss: 0.8121 - val_accuracy: 0.8475 - val_loss: 0.8155
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - accuracy: 0.8419 - loss: 0.7929✅ Tokenizer saved after epoch 4
63/63 ━━━━━━━━━━━━━━━━━━━━ 15s 235ms/step - accur

In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
import os
import pandas as pd
import re
import pickle
from google.colab import drive
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# **🔗 Mount Google Drive**
drive.mount('/content/drive')
drive_save_path = "/content/drive/MyDrive/chatbot/"
os.makedirs(drive_save_path, exist_ok=True)

# **⚡ Enable mixed precision**
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# **📝 Text Preprocessing Function**
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'s", " is", text)
    text = re.sub(r"'d", " would", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'t", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'m", " am", text)
    text = re.sub(r"[^a-zAZ0-9\s]", "", text)
    return text.strip()

# **📥 Load Skyrim Dialogue Dataset**
def load_skyrim_dialogue_dataset(file_path):
    df = pd.read_csv(file_path)
    player_inputs = df['Player Input'].astype(str).apply(preprocess_text).tolist()
    npc_responses = df['NPC Response'].astype(str).apply(preprocess_text).tolist()
    return player_inputs, npc_responses

file_path = "/content/drive/MyDrive/chatbot/dataset_refined.csv"
player_inputs, npc_responses = load_skyrim_dialogue_dataset(file_path)

# **📌 Tokenization & Padding**
max_sequence_length = 30

# Tokenizer
# Tokenizer
tokenizer = Tokenizer(oov_token="<OOV>")  # Handle unknown words
tokenizer.fit_on_texts(player_inputs + npc_responses)
vocab_size = len(tokenizer.word_index) + 1
print(f"✅ Vocabulary Size: {vocab_size}")

# Save tokenizer
tokenizer_path = os.path.join(drive_save_path, "tokenizer.pkl")
with open(tokenizer_path, "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Convert to sequences
input_sequences = pad_sequences(tokenizer.texts_to_sequences(player_inputs), maxlen=max_sequence_length, padding='post')
response_sequences = pad_sequences(tokenizer.texts_to_sequences(npc_responses), maxlen=max_sequence_length, padding='post')
y_train = np.array(response_sequences, dtype=np.int32)

# **🧪 Train-Test Split**
X_train = input_sequences

# **🛠️ LSTM Model Architecture with Attention Layer**
embedding_dim = 256
lstm_units = 512  # Increased LSTM units for more capacity

inputs = tf.keras.layers.Input(shape=(max_sequence_length,))
embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(inputs)
lstm_output = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))(embedding)

# Attention Layer (using LSTM output as both query and value)
attention = tf.keras.layers.Attention()([lstm_output, lstm_output])

dropout = tf.keras.layers.Dropout(0.5)(attention)
output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size, activation='softmax', dtype='float32'))(dropout)

model = tf.keras.models.Model(inputs=inputs, outputs=output)

# **⚙️ Compile Model with Learning Rate Schedule**
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.002,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True
)

# **⚙️ Compile Model with Learning Rate Schedule**
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),  # Use the lr_schedule directly here
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# **📌 Custom Callback to Save Tokenizer**
class SaveTokenizerCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        with open(tokenizer_path, "wb") as handle:
            pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(f"✅ Tokenizer saved after epoch {epoch + 1}")

# **🛡️ Callbacks**
best_model_path = os.path.join(drive_save_path, "skyrim_chatbot_best.keras")
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    ModelCheckpoint(best_model_path, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=0.0001),
    SaveTokenizerCallback()
]

# **🎯 Train Model**
history = model.fit(
    X_train, y_train,
    epochs=100,  # Increased epochs for better training
    batch_size=64,  # Larger batch size for stability
    callbacks=callbacks
)

# **✅ Save Final Model**
final_model_path = os.path.join(drive_save_path, "skyrim_chatbot_final.keras")
model.save(final_model_path)
print(f"✅ Model saved as {final_model_path}")

# **🗣️ Generate NPC Response Function**
def generate_response(player_input, model, tokenizer, max_sequence_length, temperature=0.3):
    input_seq = tokenizer.texts_to_sequences([player_input])
    input_seq = pad_sequences(input_seq, maxlen=max_sequence_length, padding='post')

    predicted_seq = model.predict(input_seq)[0]
    predicted_seq = np.log(predicted_seq + 1e-8) / temperature
    predicted_seq = np.exp(predicted_seq) / np.sum(np.exp(predicted_seq))
    sampled_indices = [np.random.choice(len(seq), p=seq) for seq in predicted_seq]

    response = tokenizer.sequences_to_texts([sampled_indices])[0]
    return response if response else "I am not sure, traveler."

# **🤖 Example Chatbot Interaction**
print("Welcome to the Skyrim Chatbot! Type 'exit' to leave.")
while True:
    user_input = input("You: ").strip().lower()
    if user_input in ["exit", "quit"]:
        print("NPC: Farewell, traveler!")
        break

    response = generate_response(user_input, model, tokenizer, max_sequence_length)
    print(f"NPC: {response}")

Mounted at /content/drive
✅ Vocabulary Size: 3665
Epoch 1/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - accuracy: 0.6968 - loss: 2.9926✅ Tokenizer saved after epoch 1
157/157 ━━━━━━━━━━━━━━━━━━━━ 46s 195ms/step - accuracy: 0.6969 - loss: 2.9883 - learning_rate: 0.0020
Epoch 2/100


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/callback_list.py:145: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)


157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - accuracy: 0.7285 - loss: 1.9693✅ Tokenizer saved after epoch 2
157/157 ━━━━━━━━━━━━━━━━━━━━ 32s 204ms/step - accuracy: 0.7286 - loss: 1.9693 - learning_rate: 0.0020
Epoch 3/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - accuracy: 0.7305 - loss: 1.8903✅ Tokenizer saved after epoch 3
157/157 ━━━━━━━━━━━━━━━━━━━━ 32s 203ms/step - accuracy: 0.7305 - loss: 1.8904 - learning_rate: 0.0020
Epoch 4/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.7308 - loss: 1.8424✅ Tokenizer saved after epoch 4
157/157 ━━━━━━━━━━━━━━━━━━━━ 40s 194ms/step - accuracy: 0.7308 - loss: 1.8423 - learning_rate: 0.0020
Epoch 5/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step - accuracy: 0.7358 - loss: 1.7537✅ Tokenizer saved after epoch 5
157/157 ━━━━━━━━━━━━━━━━━━━━ 42s 198ms/step - accuracy: 0.7358 - loss: 1.7539 - learning_rate: 0.0020
Epoch 6/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - accuracy: 0.7323 - loss: 1.7267✅ Tokenizer saved after epoch 6
157/

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
import os
import pandas as pd
import re
import pickle
from google.colab import drive
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# **🔗 Mount Google Drive**
drive.mount('/content/drive')
drive_save_path = "/content/drive/MyDrive/chatbot/"
os.makedirs(drive_save_path, exist_ok=True)

# **⚡ Enable mixed precision**
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# **📝 Text Preprocessing Function**
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'s", " is", text)
    text = re.sub(r"'d", " would", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'t", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'m", " am", text)
    text = re.sub(r"[^a-zAZ0-9\s]", "", text)
    return text.strip()

# **📥 Load Skyrim Dialogue Dataset**
def load_skyrim_dialogue_dataset(file_path):
    df = pd.read_csv(file_path)
    player_inputs = df['Player Input'].astype(str).apply(preprocess_text).tolist()
    npc_responses = df['NPC Response'].astype(str).apply(preprocess_text).tolist()
    return player_inputs, npc_responses

file_path = "/content/drive/MyDrive/chatbot/dataset_refined.csv"
player_inputs, npc_responses = load_skyrim_dialogue_dataset(file_path)

# **📌 Tokenization & Padding**
max_sequence_length = 30

# Tokenizer
tokenizer = Tokenizer(oov_token="<OOV>")  # Handle unknown words
tokenizer.fit_on_texts(player_inputs + npc_responses)
vocab_size = len(tokenizer.word_index) + 1
print(f"✅ Vocabulary Size: {vocab_size}")

# Save tokenizer
tokenizer_path = os.path.join(drive_save_path, "tokenizer.pkl")
with open(tokenizer_path, "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Convert to sequences
input_sequences = pad_sequences(tokenizer.texts_to_sequences(player_inputs), maxlen=max_sequence_length, padding='post')
response_sequences = pad_sequences(tokenizer.texts_to_sequences(npc_responses), maxlen=max_sequence_length, padding='post')
y_data = np.array([seq[-1] for seq in response_sequences], dtype=np.int32)  # Take only the last token
X_train, X_test, y_train, y_test = train_test_split(input_sequences, y_data, test_size=0.2, random_state=42)

y_test = np.array(y_test, dtype=np.int32)


print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


# **📥 Load Pre-trained GloVe Embeddings**
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size, embedding_dim))

glove_path = "/content/drive/MyDrive/chatbot/glove.6B.300d.txt"
embedding_dict = {}
with open(glove_path, "r", encoding="utf-8") as file:
    for line in file:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], dtype="float32")
        embedding_dict[word] = vectors

# Fill embedding matrix
for word, index in tokenizer.word_index.items():
    if word in embedding_dict:
        embedding_matrix[index] = embedding_dict[word]

# **🛠️ LSTM Model Architecture with Bahdanau Attention**
embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size,
                                            output_dim=embedding_dim,
                                            weights=[embedding_matrix],
                                            trainable=False)

# Define Attention Mechanism
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

# Define Model
inputs = tf.keras.layers.Input(shape=(max_sequence_length,))
embedding = embedding_layer(inputs)

lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True, dropout=0.3))(embedding)
query = tf.keras.layers.LSTM(512, return_sequences=False)(lstm)

attention = BahdanauAttention(512)
context_vector, _ = attention(query, lstm)

dense = tf.keras.layers.Dense(512, activation='relu')(context_vector)
outputs = tf.keras.layers.Dense(vocab_size, activation='softmax', dtype='float32')(dense)

model = tf.keras.models.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.002),
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Best model path (saving multiple checkpoints)
best_model_path = os.path.join(drive_save_path, "skyrim_chatbot_ea.keras")

# Define a custom callback to save the model and tokenizer after each epoch
class SaveModelAndTokenizerCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Save tokenizer
        tokenizer_path = os.path.join(drive_save_path, f"tokenizer.pkl")
        with open(tokenizer_path, "wb") as handle:
            pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

        # Save model
        epoch_model_path = os.path.join(drive_save_path, f"skyrim_chatbot_latest.keras")
        self.model.save(epoch_model_path)

        print(f"✅ Model and Tokenizer saved after epoch {epoch+1} at {epoch_model_path} and {tokenizer_path}")

# Callbacks including tokenizer and model saving
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    ModelCheckpoint(best_model_path, save_best_only=True, save_weights_only=False),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=0.0001),
    SaveModelAndTokenizerCallback()  # Custom callback to save model and tokenizer
]


# **Train Model**
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=callbacks
)

# **Save Final Model & Tokenizer**
final_model_path = os.path.join(drive_save_path, "skyrim_chatbot_final.keras")
model.save(final_model_path)

final_tokenizer_path = os.path.join(drive_save_path, "tokenizer_final.pkl")
with open(final_tokenizer_path, "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(f"✅ Final Model saved at {final_model_path}")
print(f"✅ Final Tokenizer saved at {final_tokenizer_path}")


# **✅ Save Final Model**
final_model_path = os.path.join(drive_save_path, "skyrim_chatbot_final.keras")
model.save(final_model_path)
print(f"✅ Model saved as {final_model_path}")

# **🗣️ Generate NPC Response Function**
def generate_response(player_input, model, tokenizer, max_sequence_length, temperature=0.3):
    input_seq = tokenizer.texts_to_sequences([player_input])
    input_seq = pad_sequences(input_seq, maxlen=max_sequence_length, padding='post')

    predicted_seq = model.predict(input_seq)[0]
    predicted_seq = np.log(predicted_seq + 1e-8) / temperature
    predicted_seq = np.exp(predicted_seq) / np.sum(np.exp(predicted_seq))
    sampled_indices = np.random.choice(len(predicted_seq), p=predicted_seq)

    response = tokenizer.sequences_to_texts([[sampled_indices]])[0]
    return response if response else "I am not sure, traveler."

# **🤖 Example Chatbot Interaction**
print("Welcome to the Skyrim Chatbot! Type 'exit' to leave.")
while True:
    user_input = input("You: ").strip().lower()
    if user_input in ["exit", "quit"]:
        print("NPC: Farewell, traveler!")
        break

    response = generate_response(user_input, model, tokenizer, max_sequence_length)
    print(f"NPC: {response}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Vocabulary Size: 3665
X_train shape: (8000, 30), y_train shape: (8000,)
X_test shape: (2000, 30), y_test shape: (2000,)
Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9249 - loss: 1.3369✅ Model and Tokenizer saved after epoch 1 at /content/drive/MyDrive/chatbot/skyrim_chatbot_latest.keras and /content/drive/MyDrive/chatbot/tokenizer.pkl
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 89ms/step - accuracy: 0.9259 - loss: 1.3213 - val_accuracy: 1.0000 - val_loss: 0.0000e+00 - learning_rate: 0.0020
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 1.0000 - loss: 0.0000e+00✅ Model and Tokenizer saved after epoch 2 at /content/drive/MyDrive/chatbot/skyrim_chatbot_latest.keras and /content/drive/MyDrive/chatbot/tokenizer.pkl
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00 

KeyboardInterrupt: 

In [ ]:
import json
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import random
from google.colab import drive  # Mount Google Drive

# **🔗 Mount Google Drive**
drive.mount('/content/drive', force_remount=True)

# **📁 Define Google Drive Path**
drive_save_path = "/content/drive/MyDrive/chatbot/"
model_path = os.path.join(drive_save_path, "skyrim_chatbot_ea.keras")
tokenizer_path = os.path.join(drive_save_path, "tokenizer.pkl")

# **📌 Define Custom Bahdanau Attention Layer (Fixed)**
@tf.keras.utils.register_keras_serializable()
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(BahdanauAttention, self).__init__(**kwargs)
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

# **📌 Load the Updated Model**
if os.path.exists(model_path):
    model = tf.keras.models.load_model(model_path, custom_objects={"BahdanauAttention": BahdanauAttention})
    print(f"✅ Model loaded from {model_path}")
else:
    raise FileNotFoundError(f"❌ Model file not found at {model_path}")

# **📌 Load the Tokenizer**
if os.path.exists(tokenizer_path):
    with open(tokenizer_path, "rb") as handle:
        tokenizer = pickle.load(handle)
    print(f"✅ Tokenizer loaded from {tokenizer_path}")
else:
    raise FileNotFoundError(f"❌ Tokenizer file not found at {tokenizer_path}")

def generate_response(input_text, max_response_length=30, temperature=0.2, top_k=5):
    max_sequence_length = 30  # Match training sequence length

    # **Tokenize and pad input**
    input_seq = pad_sequences(tokenizer.texts_to_sequences([input_text]), maxlen=max_sequence_length, padding='post')

    # **Predict output probabilities**
    prediction = model.predict(input_seq, verbose=0)

    # **🔹 Fix: Reshape Output If Needed**
    if len(prediction.shape) == 2 and prediction.shape[1] == len(tokenizer.word_index) + 1:
        prediction = prediction.reshape((1, 1, prediction.shape[1]))

    # **Validate Shape**
    if len(prediction.shape) != 3:
        raise ValueError(f"Unexpected prediction shape: {prediction.shape}. Expected (1, sequence_length, vocab_size).")

    response_text = []
    previous_words = set()

    for i in range(min(max_response_length, prediction.shape[1])):
        word_probs = prediction[0, i, :]

        # **Temperature Scaling**
        word_probs = np.exp(word_probs / temperature)
        word_probs /= np.sum(word_probs)

        # **Select Top-k Words**
        top_indices = np.argsort(word_probs)[-top_k:]
        if len(top_indices) == 0:
            continue

        word_idx = np.random.choice(top_indices, p=word_probs[top_indices] / np.sum(word_probs[top_indices]))

        if word_idx not in tokenizer.index_word:
            continue

        word = tokenizer.index_word[word_idx]

        if word in previous_words or word in ["<OOV>", "<UNK>", "<PAD>", "<END>"]:
            continue

        response_text.append(word)
        previous_words.add(word)

        if word in [".", "?", "!"]:
            break

    response = " ".join(response_text).strip()
    return response if response else "I am not sure what you mean, traveler."

# **Chatbot interaction loop**
try:
    print("Welcome to the Elder Scrolls NPC Chatbot! Type 'exit' or 'quit' to end the conversation.")
    while True:
        user_input = input("You: ").strip().lower()

        # **Exit conditions**
        if user_input in ["exit", "quit"]:
            print("NPC: Farewell, traveler. May the blessings of the Tribunal be with you!")
            break

        # **Generate and display NPC response**
        response = generate_response(user_input)
        print(f"NPC: {response}")

except KeyboardInterrupt:
    print("\nNPC: Farewell, traveler. May the blessings of the Tribunal be with you!")


Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'bahdanau_attention_5', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


✅ Model loaded from /content/drive/MyDrive/chatbot/skyrim_chatbot_ea.keras
✅ Tokenizer loaded from /content/drive/MyDrive/chatbot/tokenizer.pkl
Welcome to the Elder Scrolls NPC Chatbot! Type 'exit' or 'quit' to end the conversation.
You: hi
NPC: I am not sure what you mean, traveler.
You: hello
NPC: I am not sure what you mean, traveler.
You: fuck off
NPC: I am not sure what you mean, traveler.
You: quit
NPC: Farewell, traveler. May the blessings of the Tribunal be with you!


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
import os
import pandas as pd
import re
import pickle
from google.colab import drive
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# **🔗 Mount Google Drive**
drive.mount('/content/drive')
drive_save_path = "/content/drive/MyDrive/chatbot/"
os.makedirs(drive_save_path, exist_ok=True)

# **⚡ Enable mixed precision**
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# **📝 Text Preprocessing Function**
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'s", " is", text)
    text = re.sub(r"'d", " would", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'t", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'m", " am", text)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    return text.strip()

# **📥 Load Skyrim Dialogue Dataset**
def load_skyrim_dialogue_dataset(file_path):
    df = pd.read_csv(file_path)
    player_inputs = df['Player Input'].astype(str).apply(preprocess_text).tolist()
    npc_responses = df['NPC Response'].astype(str).apply(preprocess_text).tolist()
    return player_inputs, npc_responses

file_path = "/content/drive/MyDrive/chatbot/dataset_refined.csv"
player_inputs, npc_responses = load_skyrim_dialogue_dataset(file_path)

# **📌 Tokenization & Padding**
max_sequence_length = 30

# Tokenizer
tokenizer = Tokenizer(oov_token="<OOV>")  # Handle unknown words
tokenizer.fit_on_texts(player_inputs + npc_responses)
vocab_size = len(tokenizer.word_index) + 1
print(f"✅ Vocabulary Size: {vocab_size}")

# Save tokenizer
tokenizer_path = os.path.join(drive_save_path, "tokenizer.pkl")
with open(tokenizer_path, "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Convert to sequences
input_sequences = pad_sequences(tokenizer.texts_to_sequences(player_inputs), maxlen=max_sequence_length, padding='post')
response_sequences = pad_sequences(tokenizer.texts_to_sequences(npc_responses), maxlen=max_sequence_length, padding='post')
y_data = np.array([seq[-1] for seq in response_sequences], dtype=np.int32)  # Take only the last token
X_train, X_test, y_train, y_test = train_test_split(input_sequences, y_data, test_size=0.2, random_state=42)

y_test = np.array(y_test, dtype=np.int32)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# **📥 Load Pre-trained GloVe Embeddings**
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size, embedding_dim))

glove_path = "/content/drive/MyDrive/chatbot/glove.6B.300d.txt"
embedding_dict = {}
with open(glove_path, "r", encoding="utf-8") as file:
    for line in file:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], dtype="float32")
        embedding_dict[word] = vectors

# Fill embedding matrix
for word, index in tokenizer.word_index.items():
    if word in embedding_dict:
        embedding_matrix[index] = embedding_dict[word]

# **🛠️ LSTM Model Architecture with Bahdanau Attention**
embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size,
                                            output_dim=embedding_dim,
                                            weights=[embedding_matrix],
                                            trainable=False)

# Define Attention Mechanism
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

# **🗣️ Generate NPC Response Function**
def generate_response(player_input, model, tokenizer, max_sequence_length, temperature=0.3):
    input_seq = tokenizer.texts_to_sequences([player_input])
    input_seq = pad_sequences(input_seq, maxlen=max_sequence_length, padding='post')

    predicted_seq = model.predict(input_seq)[0]
    predicted_seq = np.log(predicted_seq + 1e-8) / temperature
    predicted_seq = np.exp(predicted_seq) / np.sum(np.exp(predicted_seq))
    sampled_indices = np.random.choice(len(predicted_seq), p=predicted_seq)

    response = tokenizer.sequences_to_texts([[sampled_indices]])[0]
    return response if response else "I am not sure, traveler."

# **🤖 Example Chatbot Interaction**
print("Welcome to the Skyrim Chatbot! Type 'exit' to leave.")
while True:
    user_input = input("You: ").strip().lower()
    if user_input in ["exit", "quit"]:
        print("NPC: Farewell, traveler!")
        break

    response = generate_response(user_input, model, tokenizer, max_sequence_length)
    print(f"NPC: {response}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Vocabulary Size: 3665
X_train shape: (8000, 30), y_train shape: (8000,)
X_test shape: (2000, 30), y_test shape: (2000,)
Welcome to the Skyrim Chatbot! Type 'exit' to leave.
You: HELLO
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step
NPC: <OOV>
You: HI
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
NPC: <OOV>


KeyboardInterrupt: Interrupted by user

In [ ]:
print(f"Input sequences shape: {input_sequences.shape}")
print(f"Response sequences shape: {response_sequences.shape}")


Input sequences shape: (10000, 30)
Response sequences shape: (10000, 30)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
import os
import pandas as pd
import re
import pickle
from google.colab import drive
import random
import nltk
from nltk.corpus import wordnet

nltk.download('wordnet')

drive.mount('/content/drive')

drive_save_path = "/content/drive/MyDrive/chatbot/"
os.makedirs(drive_save_path, exist_ok=True)
tf.keras.mixed_precision.set_global_policy('mixed_float16')

def preprocess_text(text):
    text = str(text).lower().strip()
    contractions = {
        r"n't": " not", r"'re": " are", r"'s": " is",
        r"'d": " would", r"'ll": " will", r"'t": " not",
        r"'ve": " have", r"'m": " am", r"what's": "what is",
        r"that's": "that is", r"there's": "there is"
    }
    for pat, repl in contractions.items():
        text = re.sub(pat, repl, text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def synonym_replacement(text, n=1):
    words = text.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    random.shuffle(random_word_list)
    for _ in range(n):
        for word in random_word_list:
            synonyms = [lemma.name() for syn in wordnet.synsets(word) for lemma in syn.lemmas()]
            if synonyms:
                synonym = random.choice(synonyms)
                new_words = [synonym if w == word else w for w in new_words]
                break
    return ' '.join(new_words)

def load_skyrim_dialogue_dataset(file_path, augment_data=True):
    df = pd.read_csv(file_path)
    player_inputs = df['Player Input'].astype(str).apply(preprocess_text).tolist()
    npc_responses = df['NPC Response'].astype(str).apply(preprocess_text).tolist()
    if augment_data:
        augmented_inputs, augmented_responses = [], []
        for inp, resp in zip(player_inputs, npc_responses):
            augmented_inputs.append(inp)
            augmented_responses.append(resp)
            if len(inp.split()) > 3 and len(resp.split()) > 3:
                augmented_inputs.append(synonym_replacement(inp))
                augmented_responses.append(synonym_replacement(resp))
        return augmented_inputs, augmented_responses
    return player_inputs, npc_responses

file_path = "/content/drive/MyDrive/chatbot/dataset_refined.csv"
player_inputs, npc_responses = load_skyrim_dialogue_dataset(file_path)

tokenizer = Tokenizer(oov_token="<OOV>", filters='')
tokenizer.fit_on_texts(player_inputs + npc_responses)

vocab_size = len(tokenizer.word_index) + 1
max_sequence_length = 40

input_sequences = pad_sequences(tokenizer.texts_to_sequences(player_inputs), maxlen=max_sequence_length, padding='post', truncating='post')
response_sequences = pad_sequences(tokenizer.texts_to_sequences(npc_responses), maxlen=max_sequence_length, padding='post', truncating='post')

X = np.array(input_sequences)
y = np.array(response_sequences)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

def build_model(vocab_size, embedding_dim, lstm_units, max_sequence_length):
    encoder_inputs = tf.keras.layers.Input(shape=(max_sequence_length,))
    encoder_embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)(encoder_inputs)
    encoder_lstm = tf.keras.layers.LSTM(lstm_units, return_sequences=False, return_state=True, dropout=0.2, recurrent_dropout=0.1)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

    decoder_inputs = tf.keras.layers.Input(shape=(max_sequence_length,))
    decoder_embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)(decoder_inputs)
    decoder_lstm = tf.keras.layers.LSTM(lstm_units, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.1)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])

    decoder_dense = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size, activation='softmax'))
    outputs = decoder_dense(decoder_outputs)

    return tf.keras.Model([encoder_inputs, decoder_inputs], outputs)

embedding_dim, lstm_units, learning_rate = 300, 384, 0.001
model = build_model(vocab_size, embedding_dim, lstm_units, max_sequence_length)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

class ModelAndTokenizerCheckpoint(Callback):
    def __init__(self, save_dir, tokenizer):
        super().__init__()
        self.save_dir = save_dir
        self.tokenizer = tokenizer
        os.makedirs(save_dir, exist_ok=True)
    def on_epoch_end(self, epoch, logs=None):
        model.save(os.path.join(self.save_dir, f"model_DE.keras"))
        with open(os.path.join(self.save_dir, f"tokenizer_DE.pkl"), "wb") as handle:
            pickle.dump(self.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

best_model_path = os.path.join(drive_save_path, "skyrim_chatbot_best_DE.keras")
callbacks = [EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True), ModelCheckpoint(best_model_path, save_best_only=True, monitor='val_loss'), ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5), ModelAndTokenizerCheckpoint(drive_save_path, tokenizer)]

history = model.fit([X_train, y_train], y_train, epochs=100, batch_size=256, validation_data=([X_test, y_test], y_test), callbacks=callbacks, verbose=1)

model.save(os.path.join(drive_save_path, "skyrim_chatbot_final_DE.keras"))
with open(os.path.join(drive_save_path, "tokenizer_final_DE.pkl"), "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

def generate_response(input_text, model, tokenizer, max_length):
    input_seq = pad_sequences(tokenizer.texts_to_sequences([input_text]), maxlen=max_length, padding='post')
    decoder_input = np.zeros((1, max_length))
    response = []
    for _ in range(max_length):
        output = model.predict([input_seq, decoder_input])
        predicted_id = np.argmax(output[0, -1, :])
        response.append(predicted_id)
        decoder_input[0, len(response) - 1] = predicted_id
        if predicted_id == tokenizer.word_index['<OOV>']:
            break
    return tokenizer.sequences_to_texts([response])[0]

def run_chatbot():
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ['quit', 'exit']:
            break
        print(f"NPC: {generate_response(user_input, model, tokenizer, max_sequence_length)}")

run_chatbot()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/100


OperatorNotAllowedInGraphError: Exception encountered when calling TimeDistributed.call().

[1mUsing a symbolic `tf.Tensor` as a Python `bool` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more information.[0m

Arguments received by TimeDistributed.call():
  • inputs=tf.Tensor(shape=(None, 40, 384), dtype=float16)
  • training=True
  • mask=tf.Tensor(shape=(None, 40), dtype=bool)

In [ ]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 95.1 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 w